In [2]:
import pandas as pd

forex_df = pd.read_csv("forex_dataset.csv")

In [3]:
forex_df.head()

,Year-Month,Export_Quantity_(Kg),Export_Price_(Rs.),High_Grown_Quantity_(Kg),Medium_Grown_Quantity_(Kg),Low_Grown_Quantity_(Kg),Avg_Exchange_Rate,FEE_(USD)
0,2012-01,21824945.0,497.66,5361185.05,3652353.15,14348315.90,113.942069,9.532390e+07
1,2012-02,22448624.0,509.15,5538399.31,3955745.75,13133213.00,114.102042,1.001710e+08
2,2012-03,27702254.0,536.71,6385525.09,4625138.90,17929328.00,200.000000,7.434038e+07
3,2012-04,23290994.0,540.91,7323011.00,5748401.20,16982493.50,200.000000,6.299166e+07
4,2012-05,30004885.0,567.32,8726319.70,5898297.10,18086627.25,200.000000,8.511186e+07


In [4]:
forex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year-Month                  119 non-null    object 
 1   Export_Quantity_(Kg)        119 non-null    float64
 2   Export_Price_(Rs.)          119 non-null    float64
 3   High_Grown_Quantity_(Kg)    119 non-null    float64
 4   Medium_Grown_Quantity_(Kg)  119 non-null    float64
 5   Low_Grown_Quantity_(Kg)     119 non-null    float64
 6   Avg_Exchange_Rate           119 non-null    float64
 7   FEE_(USD)                   119 non-null    float64
dtypes: float64(7), object(1)
memory usage: 7.6+ KB


In [5]:
# Create lagged features for production quantities
for lag in [1, 2]:  # t-1 and t-2
    forex_df[f'High_Grown_Quantity_t-{lag}'] = forex_df['High_Grown_Quantity_(Kg)'].shift(lag)
    forex_df[f'Medium_Grown_Quantity_t-{lag}'] = forex_df['Medium_Grown_Quantity_(Kg)'].shift(lag)
    forex_df[f'Low_Grown_Quantity_t-{lag}'] = forex_df['Low_Grown_Quantity_(Kg)'].shift(lag)

In [6]:
# Create lagged features for exported quantities and prices
for lag in [1, 2]:
    forex_df[f'Exported_Quantity_t-{lag}'] = forex_df['Export_Quantity_(Kg)'].shift(lag)
    forex_df[f'Export_Price_t-{lag}'] = forex_df['Export_Price_(Rs.)'].shift(lag)

In [7]:
# Create lagged features for exchange rates
for lag in [1, 2]:
    forex_df[f'Avg_Exchange_Rate_t-{lag}'] = forex_df['Avg_Exchange_Rate'].shift(lag)

In [8]:
# Exported-to-Production Ratio
forex_df['Exported_to_Production_Ratio'] = (
    forex_df['Export_Quantity_(Kg)'] /
    (forex_df['High_Grown_Quantity_(Kg)'] + forex_df['Medium_Grown_Quantity_(Kg)'] + forex_df['Low_Grown_Quantity_(Kg)'])
)

In [9]:
# Export Price Change
forex_df['Export_Price_Change'] = (
    (forex_df['Export_Price_(Rs.)'] - forex_df['Export_Price_(Rs.)'].shift(1)) / forex_df['Export_Price_(Rs.)'].shift(1)
)

In [10]:
forex_df.isnull().sum()

Year-Month                      0
Export_Quantity_(Kg)            0
Export_Price_(Rs.)              0
High_Grown_Quantity_(Kg)        0
Medium_Grown_Quantity_(Kg)      0
Low_Grown_Quantity_(Kg)         0
Avg_Exchange_Rate               0
FEE_(USD)                       0
High_Grown_Quantity_t-1         1
Medium_Grown_Quantity_t-1       1
Low_Grown_Quantity_t-1          1
High_Grown_Quantity_t-2         2
Medium_Grown_Quantity_t-2       2
Low_Grown_Quantity_t-2          2
Exported_Quantity_t-1           1
Export_Price_t-1                1
Exported_Quantity_t-2           2
Export_Price_t-2                2
Avg_Exchange_Rate_t-1           1
Avg_Exchange_Rate_t-2           2
Exported_to_Production_Ratio    0
Export_Price_Change             1
dtype: int64

In [11]:
# Drop rows with NaN
forex_df = forex_df.dropna().reset_index(drop=True)

In [12]:
# Convert 'Year-Month' to DateTime format
forex_df['Year-Month'] = pd.to_datetime(forex_df['Year-Month'], format='%Y-%m')

In [13]:
# Extract Year and Month
forex_df['Year'] = forex_df['Year-Month'].dt.year
forex_df['Month'] = forex_df['Year-Month'].dt.month

In [14]:
# Rearrange columns to place 'Year' and 'Month' as the first and second columns
columns_order = ['Year', 'Month'] + [col for col in forex_df.columns if col not in ['Year', 'Month']]
forex_df = forex_df[columns_order]

# Drop the 'Year-Month' column
forex_df = forex_df.drop(columns=['Year-Month', 'High_Grown_Quantity_t-1', 'Medium_Grown_Quantity_t-1', 'Low_Grown_Quantity_t-1', 
                                  'High_Grown_Quantity_t-2', 'Medium_Grown_Quantity_t-2', 'Low_Grown_Quantity_t-2', 'Exported_Quantity_t-1', 
                                  'Export_Price_t-1', 'Exported_Quantity_t-2', 'Export_Price_t-2', 'Avg_Exchange_Rate_t-1', 
                                  'Avg_Exchange_Rate_t-2', 'Exported_to_Production_Ratio', 'Export_Price_Change'])

In [15]:
forex_df.corr()

,Year,Month,Export_Quantity_(Kg),Export_Price_(Rs.),High_Grown_Quantity_(Kg),Medium_Grown_Quantity_(Kg),Low_Grown_Quantity_(Kg),Avg_Exchange_Rate,FEE_(USD)
Year,1.000000,-0.062456,-0.471996,0.911553,-0.313163,-0.179263,-0.413485,0.584302,-0.064627
Month,-0.062456,1.000000,0.294689,0.000515,-0.144114,-0.088605,0.147863,-0.042793,0.255246
Export_Quantity_(Kg),-0.471996,0.294689,1.000000,-0.436400,0.160383,0.164926,0.430112,-0.276234,0.591157
Export_Price_(Rs.),0.911553,0.000515,-0.436400,1.000000,-0.240385,-0.096796,-0.274538,0.571879,0.079289
High_Grown_Quantity_(Kg),-0.313163,-0.144114,0.160383,-0.240385,1.000000,0.891831,0.635942,-0.133737,0.026003
Medium_Grown_Quantity_(Kg),-0.179263,-0.088605,0.164926,-0.096796,0.891831,1.000000,0.668506,0.070852,-0.014657
Low_Grown_Quantity_(Kg),-0.413485,0.147863,0.430112,-0.274538,0.635942,0.668506,1.000000,-0.244294,0.302870
Avg_Exchange_Rate,0.584302,-0.042793,-0.276234,0.571879,-0.133737,0.070852,-0.244294,1.000000,-0.550686
FEE_(USD),-0.064627,0.255246,0.591157,0.079289,0.026003,-0.014657,0.302870,-0.550686,1.000000


In [16]:
# Save the final dataset
forex_df.to_csv("final_forex_dataset.csv", index=False)